<a href="https://colab.research.google.com/github/UnbreakablePACHES/Report/blob/main/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.optimize as opt
import math

# 基本ポートフォリオ
allocation = np.array([0.35,0.25,0.15,0.25])

# 期待リターン
exp_return = np.array([2.6,6,3.7,6.4])

print(allocation * exp_return)

# リスク
risk = np.array([[4.7,25.1,12.6,27.3]])



# 相関係数
R = np.mat([[1,-0.16,0.25,0.09],[-0.16,1,0.04,0.64],[0.25,0.04,1,0.57],[0.09,0.64,0.57,1]])
display(R)

[0.91  1.5   0.555 1.6  ]


matrix([[ 1.  , -0.16,  0.25,  0.09],
        [-0.16,  1.  ,  0.04,  0.64],
        [ 0.25,  0.04,  1.  ,  0.57],
        [ 0.09,  0.64,  0.57,  1.  ]])

In [ ]:
# 標準偏差行列を対角行列にする
D = np.mat([[4.7,0,0,0],
       [0,25.1,0,0],
       [0,0,12.6,0],
       [0,0,0,27.3]])
display(D)

# 共分散行列を計算する
Cov = D*R*D
display(Cov) 

d = allocation * Cov * np.array([allocation]).T
print(d)

# scalarにする
e = np.sum(d)
print(e)


matrix([[ 4.7,  0. ,  0. ,  0. ],
        [ 0. , 25.1,  0. ,  0. ],
        [ 0. ,  0. , 12.6,  0. ],
        [ 0. ,  0. ,  0. , 27.3]])

matrix([[ 22.09  , -18.8752,  14.805 ,  11.5479],
        [-18.8752, 630.01  ,  12.6504, 438.5472],
        [ 14.805 ,  12.6504, 158.76  , 196.0686],
        [ 11.5479, 438.5472, 196.0686, 745.29  ]])

[[162.9789475]]
162.97894750000006


In [ ]:
# レポート課題１
# 分散最小化モデル
def objective(w):
  return w * Cov * np.array([w]).T

# 要求収益率制約
def constraint_1(w):
  return w * exp_return 


# 空売り禁止制約と予算制約
cons=({'type':'eq','fun':lambda x: np.sum(x) - 1},
   {'type':'ineq','fun':lambda x: constraint_1(x) - 0.011},
   {'type':'ineq','fun':lambda x: x[0]},
   {'type':'ineq','fun':lambda x: x[1]},
   {'type':'ineq','fun':lambda x: x[2]},
   {'type':'ineq','fun':lambda x: x[3]})

# scipyで最適解を求める
res=opt.minimize(objective, np.array([0.25,0.25,0.25,0.25]),  method='SLSQP', 
                 constraints = cons)
print(res)

w_1 = res.x
print(w_1)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 19.569981106367923
       x: [ 9.103e-01  5.644e-02  3.150e-02  1.719e-03]
     nit: 5
     jac: [ 3.906e+01  3.906e+01  3.906e+01  8.545e+01]
    nfev: 28
    njev: 5
[0.91033267 0.0564444  0.03150418 0.00171875]


In [ ]:
# レポート課題２
# シャープレシオ最大化モデル

R_f = 0.011

sigma_y = None
# 目的関数
def objective(w):
  
  y_0 = 1/(exp_return * w - R_f)
  y = w * y_0
  obj = math.sqrt(w * Cov * np.array([w]).T) 
  return np.sum(obj)


# 制約条件
cons=({'type':'eq','fun':lambda x: np.sum(x) - 1},
   {'type':'ineq','fun':lambda x: x[0]},
   {'type':'ineq','fun':lambda x: x[1]},
   {'type':'ineq','fun':lambda x: x[2]},
   {'type':'ineq','fun':lambda x: x[3]},
   # 50%制約
   {'type':'ineq','fun':lambda x: x[2] + x[3] - x[0] - x[1] },
   # 外国債券より外国株式少ない
   {'type':'ineq','fun':lambda x: x[3] - x[2]})

res=opt.minimize(objective, np.array([0,0,0.75,0.25]),  method='SLSQP', 
                 constraints = cons)
print(res)

y_i = res.x

sigma_y = np.sum(y_i)

w_2 = y_i / sigma_y

print(w_2)


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 9.6499961139895
       x: [ 5.000e-01 -4.119e-14  2.500e-01  2.500e-01]
     nit: 2
     jac: [ 1.827e+00  1.071e+01  9.960e+00  2.499e+01]
    nfev: 10
    njev: 2
[ 5.00000000e-01 -4.11892742e-14  2.50000000e-01  2.50000000e-01]


In [ ]:
# レポート課題3
# シャープレシオ最大化モデル

R_f = 0.011

sigma_y = None
# 目的関数
def objective(w):
  
  y_0 = 1/(exp_return * w - R_f)
  y = w * y_0
  obj = math.sqrt(w * Cov * np.array([w]).T) 
  return np.sum(obj)


# 制約条件
cons=({'type':'eq','fun':lambda x: np.sum(x) - 1},
   {'type':'ineq','fun':lambda x: x[0]},
   {'type':'ineq','fun':lambda x: x[1]},
   {'type':'ineq','fun':lambda x: x[2]},
   {'type':'ineq','fun':lambda x: x[3]},

   
   # 国内株式と外国株式が等しい
   {'type':'eq','fun':lambda x: x[1] - x[3]},
   # 債券は少なくとも50％
   {'type':'ineq','fun':lambda x: -x[0] - x[2] + 0.5},
   # 外国資産は少なくとも40％
   {'type':'ineq','fun':lambda x: x[2] + x[3] - 0.4})
   
   

res=opt.minimize(objective, np.array([0,0,0.75,0.25]),  method='SLSQP', 
                 constraints = cons)
print(res)

y_i = res.x

sigma_y = np.sum(y_i)

w_2 = y_i / sigma_y

print(w_2)


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 12.766320828661138
       x: [ 3.500e-01  2.500e-01  1.500e-01  2.500e-01]
     nit: 2
     jac: [ 6.361e-01  2.056e+01  6.359e+00  2.580e+01]
    nfev: 10
    njev: 2
[0.35 0.25 0.15 0.25]
